# Визначення функцій
Натисніть, щоб розпочати

In [ ]:
!pip install ipynb
import numpy as np
import random

from text_cleaning import normalize_sentence
from custom_norm import PixelNormalization,MinibatchStdev

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:

import pickle
with open('/content/gdrive/MyDrive/Diploma/Models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

with open('/content/gdrive/MyDrive/Diploma/Models/le.pickle', 'rb') as handle:
    le = pickle.load(handle)

In [ ]:
from keras.models import load_model
model_classification = load_model('/content/gdrive/MyDrive/Diploma/Models/classification.h5', )

from tensorflow.keras.preprocessing.sequence import pad_sequences
def predict_emotion(sentence):
  sentence = normalize_sentence(sentence)
  sentence = tokenizer.texts_to_sequences([sentence])
  sentence = pad_sequences(sentence, maxlen=229, truncating='pre')
  result = le.inverse_transform(np.argmax(model_classification.predict(sentence), axis=-1))[0]
  proba =  np.max(model_classification.predict(sentence))
  return result,proba
  print(f"{result} : {proba}\n\n")


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
nltk.download('all')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

In [ ]:
def genre_selection(emotion,proba):
  match emotion:
    case "joy":#country,reggie
      if(proba>0.6):
        genre="pop"
      else: genre="reggae"
    case "sadness":
      if(proba>0.6):
        genre="blues"
      else: genre="country"
    case "anger":#metal
      if(proba>0.6):
        genre="metal"
      else: genre="metal"
    case "fear":#classical
      if(proba>0.6):
        genre="classical"
      else: genre="classical"
    case "love":#jazz disco
      if(proba>0.6):
        genre="disco"
      else: genre="jazz"
    case "surprise":#rock,hiphop energy
      if(proba>0.6):
        genre="hiphop"
      else: genre="rock"

  return genre

In [ ]:
from numpy.random import randn
import imageio
def generate_latent_points(latent_dim, n_samples):
    x_input =  randn(latent_dim * n_samples)#
    #x_input =  np.random.uniform(-1, 1, size=(latent_dim, n_samples))#до меж -1,1 uniform distribution
    x_input = x_input.reshape(n_samples, latent_dim)
    #x_input=tf.random.normal(shape=(n_samples,latent_dim))
    return x_input


def make_spectrogram(genre,model,latent_dim=100,n_samples=1):
  x=generate_latent_points(latent_dim, n_samples)
  spectrogram=model.predict(x)
  spectrogram.resize(128,192)
  #path=os.path.join('spectrogram_%s.png' % (genre))
  path="result.png"
  imageio.imwrite(path, spectrogram)






In [ ]:
import numpy as np
import librosa
import librosa.display
from scipy.signal import istft
import matplotlib.pyplot as plt
import imageio
import cv2
from scipy.io.wavfile import write

def spec_to_audio(spec):
  im = cv2.imread(spec,0)
  im = im.astype(np.uint8)
  im = im / 255.0
#im = librosa.amplitude_to_db(im, ref=0.002)
  y2 = librosa.feature.inverse.mel_to_audio(im)
  write("result.wav", 5000, y2)


In [ ]:
import os
def load_GANmodel(genre):
  path=os.path.join("/content/gdrive/MyDrive/Diploma/Models",'%s.h5' % (genre))
  cust = {'PixelNormalization': PixelNormalization, 'MinibatchStdev': MinibatchStdev}
  model  = load_model(path,cust)

  return model




# Користувацький інтерфейс

In [ ]:
#@title Введіть запит
from ipywidgets import widgets


output_text = widgets.Text()


input_text = widgets.Text(value='Запит на англійській',
    placeholder='Type something',
    description='запит:',
    disabled=False )


def bind_input_to_output(sender):
    output_text.value = input_text.value


input_text.on_submit(bind_input_to_output)

input_text

Text(value='Запит на англійській', description='запит:', placeholder='Type something')

In [ ]:
#@title Класифікація запиту
from IPython.display import display, Markdown, clear_output
button = widgets.Button(description='Класифікувати')
out = widgets.Output()
def on_button_clicked(_):

      with out:

          clear_output()
          r,g=predict_emotion(output_text.value)
          print("Ваш запит:",output_text.value)
          print("клас емоції:")
          print(f"{r} : {g}\n\n")
          genre=genre_selection(r,g)
          model=load_GANmodel(genre)
          make_spectrogram(genre,model,latent_dim=100,n_samples=1)
          print("Жанр:",genre)

button.on_click(on_button_clicked)

widgets.VBox([button,out])


In [ ]:
#@title Згенерувати музику
from IPython.display import display, Markdown, clear_output
from matplotlib import pyplot as plt
import tensorflow as tf
button3 = widgets.Button(description='Згенерувати музику')
out = widgets.Output()




def on_button_clicked3(_):

      with out:

          clear_output()
          img = tf.keras.preprocessing.image.load_img('result.png')
          plt.imshow(img)
          spec_to_audio("result.png")
          #Audio("result.wav")

button3.on_click(on_button_clicked3)

widgets.VBox([button3,out])

In [ ]:
#@title Натисніть, щоб прослухати
Audio("result.wav")